In [11]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [12]:
df = pd.read_csv(r'/content/drive/MyDrive/Sentiment.csv')

# New section

In [7]:
print("Szie of the data : ", df.shape)

Szie of the data :  (13871, 21)


In [8]:
df_filtered = df[df["sentiment"]!='Neutral']
df_filtered.shape

(10729, 21)

In [9]:
df_filtered

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
5,6,Ted Cruz,0.6332,yes,1.0000,Positive,0.6332,None of the above,1.0000,NaN,...,NaN,228,NaN,NaN,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",NaN,2015-08-07 09:54:44 -0700,629697194283499520,NaN,Central Time (US & Canada)
6,7,No candidate mentioned,1.0000,yes,1.0000,Negative,0.6761,FOX News or Moderators,1.0000,NaN,...,NaN,17,NaN,NaN,RT @warriorwoman91: I liked her and was happy ...,NaN,2015-08-07 09:54:44 -0700,629697192383672320,North Georgia,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [52]:
# Q1 total number of positive and negative sentiments.

In [51]:
df_filtered2['sentiment'].value_counts()

Negative    8493
Positive    2236
Name: sentiment, dtype: int64

In [54]:
df_filtered["sentiment"] =np.where(df_filtered["sentiment"].str.contains("Positive"), 1, 0)

In [55]:
df_filtered2=df_filtered[['sentiment','text']]
df_filtered2

,sentiment,text
1,1,RT @ScottWalker: Didn't catch the full #GOPdeb...
3,1,RT @RobGeorge: That Carly Fiorina is trending ...
4,1,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
5,1,"RT @GregAbbott_TX: @TedCruz: ""On my first day ..."
6,0,RT @warriorwoman91: I liked her and was happy ...
...,...,...
13866,0,RT @cappy_yarbrough: Love to see men who will ...
13867,1,RT @georgehenryw: Who thought Huckabee exceede...
13868,1,"RT @Lrihendry: #TedCruz As President, I will a..."
13869,0,RT @JRehling: #GOPDebate Donald Trump says tha...


In [15]:
#lower caseing
df_filtered2['text'] = df_filtered2['text'].apply(lambda x: x.lower()) 


In [16]:
# removing special characters
df_filtered2['text'] = df_filtered2['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) 


In [17]:
# removing rt
df_filtered2['text'] = df_filtered2.text.str.replace('rt','') 

In [56]:
train_df, test_df = train_test_split(df_filtered2, test_size = 0.2, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (8583, 2)
Test data size :  (2146, 2)


In [57]:
top_words = 6000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['text'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['text'])

max_review_length = 130
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['sentiment']

In [43]:
# Q2 sequential LSTM 

In [58]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 130, 32)           192032    
                                                                 
 lstm_4 (LSTM)               (None, 100)               53200     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 245,333
Trainable params: 245,333
Non-trainable params: 0
_________________________________________________________________


In [61]:
hist=model.fit(X_train,y_train, epochs=8, batch_size=64, validation_split=0.2)

Epoch 1/8
108/108 [==============================] - 16s 146ms/step - loss: 0.1871 - accuracy: 0.9244 - val_loss: 0.4040 - val_accuracy: 0.8486
Epoch 2/8
108/108 [==============================] - 16s 148ms/step - loss: 0.1510 - accuracy: 0.9404 - val_loss: 0.4296 - val_accuracy: 0.8550
Epoch 3/8
108/108 [==============================] - 16s 148ms/step - loss: 0.1220 - accuracy: 0.9551 - val_loss: 0.4959 - val_accuracy: 0.8614
Epoch 4/8
108/108 [==============================] - 16s 147ms/step - loss: 0.1039 - accuracy: 0.9581 - val_loss: 0.5037 - val_accuracy: 0.8509
Epoch 5/8
108/108 [==============================] - 16s 146ms/step - loss: 0.0924 - accuracy: 0.9634 - val_loss: 0.5603 - val_accuracy: 0.8497
Epoch 6/8
108/108 [==============================] - 16s 147ms/step - loss: 0.0914 - accuracy: 0.9659 - val_loss: 0.5387 - val_accuracy: 0.8497
Epoch 7/8
108/108 [==============================] - 16s 147ms/step - loss: 0.0795 - accuracy: 0.9699 - val_loss: 0.5511 - val_accuracy:

In [42]:
# Q3 check the sentiment for the following two sentences  

In [62]:
sentence={
    "'He is a great leader",
    "He is a terrible leader."
}

In [63]:
list_tokenized_test = tokenizer.texts_to_sequences(sentence)
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)

In [64]:
prediction = model.predict(X_test)

In [65]:
prediction

array([[0.94680214],
       [0.09080806]], dtype=float32)